# AMOS 2024 - Heatwave case study using BARRA2 reanalysis

##### In this notebook we demonstrate the use of BARRA2 data in the NCI Data Collection to explore the NSW heatwave of January 2017.

##### More information on BARRA2 data: https://opus.nci.org.au/pages/viewpage.action?pageId=264241166

##### Before using this notebook, users must join ob53 project via, https://my.nci.org.au/mancini/project/ob53/join

In [1]:
import os, sys
os.chdir("/g/data/hd50/chs548/BARRA2_evaluation/jt/notebooks/")
import loaddata
from datetime import datetime as dt
# Here we will use the LOADDATA module to simply the loading of the data
import loaddata
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import numpy as np
import xarray as xr

## Explore the data sets - what's available and what they are

In [4]:
# What experiments are available in BARRA2 at this time?
loaddata.list_experiments("BARRA2")

# At this time, we have BARRA-R2 12 km reanalysis data. 
# Later this year, the BARRA-RE2 ensemble and BARRA-C2 4.4 km reanalysis data will be available.

{domain}   {era5_mem}   {model}
AUS-11   hres   BARRA-R2


In [8]:
# What hourly variables are available?
_ = loaddata.list_barra2_variables("BARRA-R2", "1hr")

rsut, rluscs, ta1500m, va400, pr, ta200m, ta700, clwvi, hus500, tas, uasmean, mrsos, ua400, zg925, ta250m, rlus, prc, va150m, ua300, zg300, zg400, zg850, ua850, va250m, va200m, ta600, rsdscs, wa300, ua50m, hus200, ps, ta50m, omega500, prsn, va600, tasmean, cll, hus400, vasmean, wa700, wsgsmax, uasmax, vasmax, ua250m, CAPE, va850, ua700, ua1000, hus1000, ua200m, va700, hus950, ua150m, ua1500m, prw, va300, wa1000, ua600, ua100m, ua925, zg200, wa600, va100m, va200, rsutcs, rsds, evspsblpot, hus600, zg1000, ua500, sfcWind, mrfsos, hfls, ta850, ta100m, ua200, rsdt, clh, va925, zg600, hus300, zg500, hurs, psl, ts, vas, huss, rldscs, ta925, ta150m, wa500, va1500m, uas, hus700, ta500, zg700, clivi, wa200, va50m, ta200, rlutcs, tasmin, ta300, clm, CIN, ta400, rlds, wa400, zmla, va1000, rsdsdir, rsuscs, hfss, rlut, wa850, ta1000, va500, clt, ta950, wa925, tasmax, hus850, rsus, hus925


In [9]:
# For heatwave case study, we will look at the screen-level temperature, near-surface wind vector and low level cloud cover
# First check they are what they are, and explore different variable options
_ = loaddata.whatis("1hr", "tas")

Short name: tas
long_name: Near-Surface Air Temperature
standard_name: air_temperature
units: K
cell_methods: time: point (interval: 1H)


In [10]:
_ = loaddata.whatis("1hr", "tasmax")

Short name: tasmax
long_name: Hourly Maximum Near-Surface Air Temperature
standard_name: air_temperature
units: K
cell_methods: time: maximum (interval: 1 hour)


In [14]:
_ = loaddata.whatis("1hr", "uas")

Short name: uas
long_name: Eastward Near-Surface Wind
standard_name: eastward_wind
units: m s-1
cell_methods: time: point (interval: 1H) area: interpolation (method: bilinear)


In [16]:
_ = loaddata.whatis("1hr", "uasmax")

Short name: uasmax
long_name: Eastward Near-Surface Wind
standard_name: eastward_wind
units: m s-1
cell_methods: area: interpolation (method: bilinear) time: maximum (interval: 1H)


In [12]:
_ = loaddata.whatis("1hr", "vas")

Short name: vas
long_name: Northward Near-Surface Wind
standard_name: northward_wind
units: m s-1
cell_methods: time: point (interval: 1H) area: interpolation (method: bilinear)


In [109]:
_ = loaddata.whatis("1hr", "cll")

Short name: cll
long_name: Low Level Cloud Fraction
standard_name: cloud_area_fraction_in_atmosphere_layer
units: %
cell_methods: time: mean (interval: 1 hour)


In [37]:
_ = loaddata.whatis("fx", "orog")

Short name: orog
long_name: Surface Altitude
standard_name: surface_altitude
units: m
frequency: fx


In [131]:
_ = loaddata.whatis("fx", "sftlf")

Short name: sftlf
long_name: Percentage of the grid  cell occupied by land (including lakes)
standard_name: land_area_fraction
units: %
frequency: fx


## Function to load the data

In [3]:
# We will use hourly maximum temperature (tasmax), hourly inst u and v (uas, vas), and hourly mean low-level cloud cover (cll)
# But feel free to explore other variables of your interests, such as pressure-level variables such as va850, ta850
help(loaddata.load_barra2_data)

Help on function load_barra2_data in module loaddata:

load_barra2_data(model, freq, variable, version='*', tstart=None, tend=None, loc=None, latrange=None, lonrange=None)
    Returns the BARRA2 data
    
    Parameters:
       model (str): Model, e.g., BARRA-R2, BARRA-RE2, BARRA-C2
       freq (str): Time frequency of the data, e.g. 1hr, day, mon
       variable (str): Variable name, e.g., tas, uas, pr
       version (str): Data release version if multiple available
       tstart (datetime.datetime): Start of the time range
       tend (datetime.datetime): End of the time range
       loc (tuple of float), (latitude, longitude) if requesting data closest to a point location
       latrange (tuple of float), (latmin, latmax) if requesting data over a latitude range
       lonrange (tuple of float), (lonmin, lonmax) if requesting data over a longitude range
    
    Returns:
       data (xarray.Dataset): Extracted data
       
    Note: model, freq, variable as per labels in
        /g/

In [7]:
# What's the temperature in the region around Sydney during January 2017?

loc = (-33.8688, 151.2093)
latmin_c = loc[0] - 1
latmax_c = loc[0] + 1
lonmin_c = loc[1] - 1
lonmax_c = loc[1] + 1
tstart = dt(2016, 12, 1)
tend = dt(2017, 4, 1)

# Load tasmax data
ds_tasmax = loaddata.load_barra2_data("BARRA-R2", "1hr", "tasmax", 
                                      tstart=tstart, tend=tend,
                                      latrange=(latmin_c, latmax_c),
                                      lonrange=(lonmin_c, lonmax_c))
# Convert to degC
ds_tasmax['tasmax'] = ds_tasmax['tasmax'] - 273.15
(NT, NY, NX) = ds_tasmax['tasmax'].shape

# Load land sea mask
ds_lsm = loaddata.load_barra2_data("BARRA-R2", "fx", "sftlf",
                                   latrange=(latmin_c, latmax_c),
                                      lonrange=(lonmin_c, lonmax_c))

# Compute spatial mean of tasmax over land, over the smaller subdomain around Sydney
mask = (np.tile(ds_lsm['sftlf'].values, (NT,1,1)) >= 100)
da_tasmax_spatial_av = ds_tasmax['tasmax'].where(mask).mean(dim=['lat', 'lon'])

In [ ]:
fig = plt.figure(figsize=(10, 3))

# 
t0 = dt(2017, 2, 7)
t1 = dt(2017, 2, 14)
#
# First subfigure showing the temperature changes during the whole period
# 
ax = plt.subplot(1, 2, 1)
da_tasmax_spatial_av.plot.line()
plt.axvline(x=t0, color='r')
plt.axvline(x=t1, color='r')
#
# Second subfigure showing the temperature changes 
# 
ax = plt.subplot(1, 2, 2)
da_tasmax_spatial_av.sel(time=slice(t0, t1)).plot.line()

In [18]:
# Define domain and time of interests as per above
tstart = dt(2017, 2, 8)
tend = dt(2017, 2, 14)

latmin_r = loc[0] - 5
latmax_r = loc[0] + 5
lonmin_r = loc[1] - 5
lonmax_r = loc[1] + 5

In [19]:
# Load the orography data
ds_orog = loaddata.load_barra2_data("BARRA-R2", "fx", "orog", 
                                    latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r))
# Load land sea mask
ds_lsm = loaddata.load_barra2_data("BARRA-R2", "fx", "sftlf",
                                   latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r))

In [20]:
ds_tasmax = loaddata.load_barra2_data("BARRA-R2", "1hr", "tasmax", 
                                      tstart=tstart, tend=tend,
                                      latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r))
ds_uas = loaddata.load_barra2_data("BARRA-R2", "1hr", "uas", 
                                      tstart=tstart, tend=tend,
                                      latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r))
ds_vas = loaddata.load_barra2_data("BARRA-R2", "1hr", "vas", 
                                      tstart=tstart, tend=tend,
                                      latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r))
ds_cll = loaddata.load_barra2_data("BARRA-R2", "1hr", "cll", 
                                      tstart=tstart, tend=tend,
                                      latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r))

# Convert temperature from K to degC
ds_tasmax['tasmax' ] = ds_tasmax['tasmax'] - 273.15
(NT, NY, NX) = ds_tasmax['tasmax'].shape

mask = (np.tile(ds_lsm['sftlf'].values, (NT,1,1)) >= 100)

In [21]:
# Combine uas and vas into a single xr.Dataset object
ds_uv = xr.merge([ds_uas['uas'], ds_vas['vas']])
ds_uv_subsampled = ds_uv.isel(lat=range(0, NY, 7), lon=range(0, NX, 7)).compute()
#ds_uv_subsampled = ds_uv #.isel(lat=range(0, NY, 10), lon=range(0, NX, 10)).compute()

In [22]:
# Compute spatial mean of tasmax over the smaller subdomain around Sydney
ds_lsm_c = ds_lsm.sel(lat=slice(latmin_c, latmax_c), lon=slice(lonmin_c, lonmax_c))
mask_c = (np.tile(ds_lsm_c['sftlf'], (NT,1,1)) >= 100)
ds_tasmax_c = ds_tasmax.sel(lat=slice(latmin_c, latmax_c), lon=slice(lonmin_c, lonmax_c))
da_tasmax_spatial_av = ds_tasmax_c['tasmax'].where(mask_c).mean(dim=['lat', 'lon'])

In [ ]:
for time_step in range(0, NT, 3):
    
    # Plotting for this time step
    t = ds_tasmax['time'][time_step]
    
    # Set up the figure object
    fig = plt.figure(figsize=(14, 4))
    
    #
    # First subfigure showing the timeseries of spatial mean tasmax
    #
    ax1 = plt.subplot(1, 3, 1)
    da_tasmax_spatial_av.plot.line()
    ax1.plot(t.data, da_tasmax_spatial_av.data[time_step], 'or')
    # label the time step
    ax1.set_title(t.data)
    
    #
    # Second subfigure plotting temperature and wind vectors
    #
    ax2 = plt.subplot(1, 3, 2)
    # Plot tasmax as background
    ds_tasmax.sel(time=t, method='nearest')['tasmax'].plot(vmin=0, vmax=40, cmap=mpl.cm.RdBu_r, cbar_kwargs={"shrink": 0.5})
    # Plot the 10m wind vector
    ds_uv_subsampled.sel(time=t, method='nearest').plot.quiver(x='lon', y='lat', u='uas', v='vas', color='blue')
    # Plot surface altitude as contour
    ds_orog['orog'].plot.contour(levels=4, colors='k')
    # Indicate where is Sydney
    ax2.plot(loc[1], loc[0], 'xr', markersize=15)
    
    #
    # Last subfigure plotting low level cloud
    #
    ax3 = plt.subplot(1, 3, 3)
    # Plot cloud 
    ds_cll['cll'].sel(time=t, method='nearest').plot(vmin=0, vmax=100, cmap=mpl.cm.Greys_r, cbar_kwargs={"shrink": 0.5})
    # Plot surface altitude as contour
    ds_orog['orog'].plot.contour(levels=4, colors='b')
    # Indicate where is Sydney
    ax3.plot(loc[1], loc[0], 'xr', markersize=15)
    
    fig.tight_layout()
    
# this may take a while!